A big thanks to <a href="https://github.com/ageron/handson-ml2/blob/master/09_unsupervised_learning.ipynb">Aurélien Geron</a> and the team at Tensorflow for providing great [tutorials](https://www.tensorflow.org/tutorials). Most of the code below is adapted from their code for this use case.

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/thekizoch/ml/blob/master/notebooks/semi-supervised-cnn-with-kmeans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
  <td>
    <a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/thekizoch/ml/blob/master/notebooks/semi-supervised-cnn-with-kmeans.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" /></a>
  </td>
</table>

# Semi-supervised CNN with K-means clustering

## setup

In [1]:
import numpy as np
import tensorflow as tf
import urllib.request
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

## load mnist data

In [2]:
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
mnist.target = mnist.target.astype(np.int64)

X_train, X_test, y_train, y_test = train_test_split(
    mnist["data"], mnist["target"], random_state=42)

In [3]:
# scale data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [4]:
# shape of training data
X_train.shape

(52500, 784)

## K-means clustering: manually labelled representative images

We run a basic K-means clustering model from [scikit-learn](https://scikit-learn.org/stable/index.html) with 50 clusters on 52,500 training examples.

Note this step takes over 5 minutes to run on Colab, due to our dataset size. This is much better than manually labelling over 50,000 examples.

In [5]:
k = 50
kmeans = KMeans(n_clusters=k, random_state=42)
X_digits_dist = kmeans.fit_transform(X_train)
# show shape
X_digits_dist.shape

(52500, 50)

Now we will choose the centroids of the 50 clusters as the representative images. These 50 images, in a real world application, we would label manually.

In [ ]:
# get centroid images
representative_digit_idx = np.argmin(X_digits_dist, axis=0)
X_representative_digits = X_train[representative_digit_idx]
# plot
plt.figure(figsize=(8, 2))
for index, X_representative_digit in enumerate(X_representative_digits):
    plt.subplot(k // 10, 10, index + 1)
    plt.imshow(X_representative_digit.reshape(28, 28), cmap="binary", interpolation="bilinear")
    plt.axis('off')

plt.show()

In this case where the train data already contains the label, so we will just copy the labels over to save time.

In [ ]:
# confirm with image above
y_representative_digits = y_train[representative_digit_idx]
y_train[representative_digit_idx]

## partial label propagation

With these 50 representative labelled examples, we will label 50% of that training data that lies closest to each centroid. This is because examples that lie far from centroids are more likelly to be misclassified. 

If we did 100%, we would call this full label propagation.

This percentile can be optimized, but that's out of scope for this exercise.

In [ ]:
percentile_closest = 50

# define fully propagated y
y_train_propagated = np.empty(len(X_train), dtype=np.int32)
for i in range(k):
    y_train_propagated[kmeans.labels_==i] = y_representative_digits[i]

# define partial label progation X and y
X_cluster_dist = X_digits_dist[np.arange(len(X_train)), kmeans.labels_]
for i in range(k):
    in_cluster = (kmeans.labels_ == i)
    cluster_dist = X_cluster_dist[in_cluster]
    cutoff_distance = np.percentile(cluster_dist, percentile_closest)
    above_cutoff = (X_cluster_dist > cutoff_distance)
    X_cluster_dist[in_cluster & above_cutoff] = -1

partially_propagated = (X_cluster_dist != -1)

X_train_partially_propagated = X_train[partially_propagated]
y_train_partially_propagated = y_train_propagated[partially_propagated]

X_train_partially_propagated.shape

## CNN on Mnist with semi-supervised learning

Before we use the 39,370 training examples with propagated labels, we need to get a baseline for how a CNN model would perform on only 50 random labeled examples.

To speed development, we will use the Mnist dataset from Tensorflow and the [Keras Sequential API](https://www.tensorflow.org/guide/keras/overview)

## base line: CNN on only 50 examples

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0
# verify size
train_images.shape

In [ ]:
plt.figure(figsize=(10,10))
for i in range(10):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i])
plt.show()

In [ ]:
# reshape
train_images = train_images.reshape(train_images.shape + (1,))
test_images = test_images.reshape(test_images.shape + (1,))
train_images.shape

## model

We build a simple CNN model architecture, which we will use for training on

*   50 random labelled training images 
*   39,370 training images propagated from 50 labelled representative training images

In [ ]:
def get_model():
  model = models.Sequential()
  model.add(layers.Conv2D(8, (3, 3), activation='relu', input_shape=(28, 28, 1)))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(16, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(32, (3, 3), activation='relu'))
  # add dnn
  model.add(layers.Flatten())
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(10))
  return model

## train and evaluate on 50 

In [ ]:
# train
train_num_examples = 50


model = get_model()
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images[:train_num_examples], train_labels[:train_num_examples], epochs=10, 
                    validation_data=(test_images, test_labels))

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.0, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print(f'test accuracy: {test_acc}')

We're not able to break 30% accuracy on examples the model has never seen before, using only 50 training examples.

## train and evaluate on propagated examples

We'll quickly verify that our data is in the same shape, and then train/evaluate. 

To save time, we'll only train for 5 epochs. Above we ran for 10 epochs.

In [ ]:
X_train_partially_propagated = X_train_partially_propagated.reshape(
    X_train_partially_propagated.shape[0],28,28)

plt.figure(figsize=(10,10))
for i in range(10):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_train_partially_propagated[i])
    # The CIFAR labels happen to be arrays, 
    # which is why you need the extra index
    #plt.xlabel(class_names[train_labels[i][0]])
plt.show()

In [ ]:
# reshape
X_train_partially_propagated = X_train_partially_propagated.reshape(
    X_train_partially_propagated.shape + (1,))
X_train_partially_propagated.shape

In [ ]:
# train
model = get_model()
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(X_train_partially_propagated, y_train_partially_propagated, epochs=5, 
                    validation_data=(test_images, test_labels))

In [ ]:
# evaluate
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print(f'test accuracy: {test_acc}')

## discussion, further investigation

Using the partially propagated examples, were we able to boost test accuracy from 30% to 84%. 

This was done without any optimization on

* k (number of clusters)
* percentage of partial propagation
* CNN Model architecture

Most of the data in the world is unlabelled. Using the unsupervised learning algorithm of k-means clustering, we were able to boost accuracy considerably with a low requirement of labelled data.